In [ ]:
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer  
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, GRU, Dropout,Embedding ,Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
df = df[['target','comment_text']]
df.head()

In [ ]:
x = df['comment_text'].tolist()
x[:5]

In [ ]:
y = df['target'].tolist()
y[:5]

In [ ]:
max_tokens = 10000
maxlen=100
x_train, x_test, y_train, y_test = train_test_split(x[:60000], y[:60000], test_size=0.2, random_state=42)
tok = Tokenizer(num_words=max_tokens, oov_token='UNK')
tok.fit_on_texts(x)
x_train = tok.texts_to_sequences(x_train)
x_test = tok.texts_to_sequences(x_test )
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [ ]:
model = Sequential()
model.add(Embedding(max_tokens, 50, input_length=maxlen))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
y_train= np.array (y_train)
y_test= np.array(y_test)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train, y_train,batch_size=64, epochs=7,validation_data=(x_test, y_test))

In [ ]:
test_df= pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
test_df= test_df['comment_text']
test_df.head()

In [ ]:
xt = test_df.tolist()
xt[:5]

In [ ]:
vocab_sz = 10000
maxlen=100
tok = Tokenizer(num_words=vocab_sz, oov_token='UNK')
tok.fit_on_texts(xt)
xt = tok.texts_to_sequences(xt)
xt = pad_sequences(xt, maxlen=maxlen)

In [ ]:
y_pred= model.predict(xt)


In [ ]:
y_pred = np.where(y_pred>=0.5,1,0)
df1 = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
df1['prediction'] = y_pred
df1.head()